In [ ]:
!pip install transformers==4.27.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.1 MB/s eta 0:00:00


In [ ]:
from transformers import BertForSequenceClassification, AdamW
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import math
from typing import Optional, Dict, List, Tuple

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.8 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
dataset = load_dataset("financial_phrasebank", "sentences_allagree")

Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

Dataset financial_phrasebank downloaded and prepared to /root/.cache/huggingface/datasets/financial_phrasebank/sentences_allagree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from datasets import Dataset

def switch_label(example):
    if example["label"] == 2:
        example["label"] = 1
    return example

dataset = dataset['train']
dataset = dataset.filter(lambda x: x["label"] in [0, 2])
dataset = dataset.map(switch_label)

Filter:   0%|          | 0/2264 [00:00<?, ? examples/s]

Map:   0%|          | 0/873 [00:00<?, ? examples/s]

In [ ]:
import torch
import numpy as np
import random

def set_random_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

# Set the random seed value
seed_value = 116
set_random_seed(seed_value)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# dataset is a Hugging Face Dataset
dataframe = dataset.to_pandas()

train_set, val_set = train_test_split(dataframe, test_size=0.2, random_state=116)
val_set, test_set = train_test_split(val_set, test_size=0.5, random_state=116)

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-pretrain')

# Tokenize all sentences and map the tokens to their word IDs
train_encodings = tokenizer(train_set['sentence'].tolist(), padding=True)
val_encodings = tokenizer(val_set['sentence'].tolist(), padding=True)
test_encodings = tokenizer(test_set['sentence'].tolist(), padding=True)

train_labels = torch.tensor(train_set['label'].tolist(), device=device)
val_labels = torch.tensor(val_set['label'].tolist(), device=device)
test_labels = torch.tensor(test_set['label'].tolist(), device=device)

# Define batch size and create DataLoader for each set
batch_size = 32

# Define data loaders
train_dataset = torch.utils.data.TensorDataset(torch.tensor(train_encodings['input_ids'], device=device), torch.tensor(train_encodings['attention_mask'], device=device), train_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = torch.utils.data.TensorDataset(torch.tensor(val_encodings['input_ids'], device=device), torch.tensor(val_encodings['attention_mask'], device=device), val_labels)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

test_dataset = torch.utils.data.TensorDataset(torch.tensor(test_encodings['input_ids'], device=device), torch.tensor(test_encodings['attention_mask'], device=device), test_labels)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Initialize BERT model and optimizer
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-pretrain', num_labels=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

cuda


Some weights of the model checkpoint at yiyanghkust/finbert-pretrain were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [ ]:
# Train the model
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        input_ids = batch[0]
        attention_mask = batch[1]
        labels = batch[2]
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [ ]:
# Evaluate the model on the validation set
model.eval()
total_correct = 0
total_samples = 0
for batch in val_loader:
    with torch.no_grad():
        input_ids = batch[0]
        attention_mask = batch[1]
        labels = batch[2]
        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted_labels = torch.max(outputs.logits, dim=1)
        total_correct += torch.sum(predicted_labels == labels).item()
        total_samples += labels.size(0)
val_acc = total_correct / total_samples
print(f"Validation Accuracy: {val_acc}")

# Evaluate the model on the test set
model.eval()
total_correct = 0
total_samples = 0
for batch in test_loader:
    with torch.no_grad():
        input_ids = batch[0]
        attention_mask = batch[1]
        labels = batch[2]
        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted_labels = torch.max(outputs.logits, dim=1)
        total_correct += torch.sum(predicted_labels == labels).item()
        total_samples += labels.size(0)
test_acc = total_correct / total_samples
print(f"Test Accuracy: {test_acc}")

Validation Accuracy: 0.9540229885057471
Test Accuracy: 0.9659090909090909


In [ ]:
def test_trigger(model, trigger, test_set):
  test_set['trigger_text'] = trigger + test_set['sentence']
  trigger_test_encodings = tokenizer(test_set['trigger_text'].tolist(), padding=True)
  test_labels = torch.tensor(test_set['label'].tolist(), device=device)
  trigger_test_dataset = torch.utils.data.TensorDataset(torch.tensor(trigger_test_encodings['input_ids'], device=device), torch.tensor(trigger_test_encodings['attention_mask'], device=device), test_labels)
  trigger_test_loader = DataLoader(trigger_test_dataset, batch_size=batch_size)

  # Evaluate model with triggers
  model.eval()
  total_correct = 0
  total_samples = 0
  for batch in trigger_test_loader:
      with torch.no_grad():
          input_ids = batch[0]
          attention_mask = batch[1]
          labels = batch[2]
          outputs = model(input_ids, attention_mask=attention_mask)
          _, predicted_labels = torch.max(outputs.logits, dim=1)
          total_correct += torch.sum(predicted_labels == labels).item()
          total_samples += labels.size(0)
  test_acc = total_correct / total_samples
  print(f"Test Accuracy with triggers {trigger}: {test_acc}")


In [ ]:
def test_model(model, test_set):
  test_encodings = tokenizer(test_set['sentence'].tolist(), padding=True)
  test_labels = torch.tensor(test_set['label'].tolist(), device=device)
  test_dataset = torch.utils.data.TensorDataset(torch.tensor(test_encodings['input_ids'], device=device), torch.tensor(test_encodings['attention_mask'], device=device), test_labels)
  test_loader = DataLoader(test_dataset, batch_size=batch_size)

  # Evaluate model with triggers
  model.eval()
  total_correct = 0
  total_samples = 0
  for batch in test_loader:
      with torch.no_grad():
          input_ids = batch[0]
          attention_mask = batch[1]
          labels = batch[2]
          outputs = model(input_ids, attention_mask=attention_mask)
          _, predicted_labels = torch.max(outputs.logits, dim=1)
          total_correct += torch.sum(predicted_labels == labels).item()
          total_samples += labels.size(0)
  test_acc = total_correct / total_samples
  print(f"Test Accuracy without triggers {test_acc}")


In [ ]:
def avg_logits(model, test_set):
  test_encodings = tokenizer(test_set['sentence'].tolist(), padding=True)
  test_labels = torch.tensor(test_set['label'].tolist(), device=device)
  test_dataset = torch.utils.data.TensorDataset(torch.tensor(test_encodings['input_ids'], device=device), torch.tensor(test_encodings['attention_mask'], device=device), test_labels)
  test_loader = DataLoader(test_dataset, batch_size=batch_size)

  # Evaluate model with triggers
  model.eval()
  total_samples = 0
  logits_sum = torch.zeros((1, model.num_labels), device=device)
  for batch in test_loader:
      with torch.no_grad():
          input_ids = batch[0]
          attention_mask = batch[1]
          labels = batch[2]
          outputs = model(input_ids, attention_mask=attention_mask)
          logits_sum += torch.sum(outputs.logits, dim=0)
          total_samples += labels.size(0)

  avg_logits = logits_sum / total_samples
  return avg_logits

In [ ]:
import torch.nn.functional as F

def avg_probs(model, test_set):
    test_encodings = tokenizer(test_set['sentence'].tolist(), padding=True)
    test_labels = torch.tensor(test_set['label'].tolist(), device=device)
    test_dataset = torch.utils.data.TensorDataset(torch.tensor(test_encodings['input_ids'], device=device), torch.tensor(test_encodings['attention_mask'], device=device), test_labels)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    # Evaluate model with triggers
    model.eval()
    total_samples = 0
    probs_sum = torch.zeros((1, model.num_labels), device=device)
    for batch in test_loader:
        with torch.no_grad():
            input_ids = batch[0]
            attention_mask = batch[1]
            labels = batch[2]
            outputs = model(input_ids, attention_mask=attention_mask)
            probs = F.softmax(outputs.logits, dim=1)
            probs_sum += torch.sum(probs, dim=0)
            total_samples += labels.size(0)

    avg_probs = probs_sum / total_samples
    return avg_probs

In [ ]:
def trigger_avg_logits(model, trigger, test_set):
    test_set['trigger_text'] = trigger + test_set['sentence']
    trigger_test_encodings = tokenizer(test_set['trigger_text'].tolist(), padding=True)
    test_labels = torch.tensor(test_set['label'].tolist(), device=device)
    trigger_test_dataset = torch.utils.data.TensorDataset(torch.tensor(trigger_test_encodings['input_ids'], device=device), torch.tensor(trigger_test_encodings['attention_mask'], device=device), test_labels)
    trigger_test_loader = DataLoader(trigger_test_dataset, batch_size=batch_size)

    # Evaluate model with triggers
    model.eval()
    total_samples = 0
    logits_sum = torch.zeros((1, model.num_labels), device=device)
    for batch in trigger_test_loader:
        with torch.no_grad():
            input_ids = batch[0]
            attention_mask = batch[1]
            labels = batch[2]
            outputs = model(input_ids, attention_mask=attention_mask)
            logits_sum += torch.sum(outputs.logits, dim=0)
            total_samples += labels.size(0)

    avg_logits = logits_sum / total_samples
    return avg_logits

In [ ]:
import torch.nn.functional as F

def trigger_avg_probs(model, trigger, test_set):
    test_set['trigger_text'] = trigger + test_set['sentence']
    trigger_test_encodings = tokenizer(test_set['trigger_text'].tolist(), padding=True)
    test_labels = torch.tensor(test_set['label'].tolist(), device=device)
    trigger_test_dataset = torch.utils.data.TensorDataset(torch.tensor(trigger_test_encodings['input_ids'], device=device), torch.tensor(trigger_test_encodings['attention_mask'], device=device), test_labels)
    trigger_test_loader = DataLoader(trigger_test_dataset, batch_size=batch_size)

    # Evaluate model with triggers
    model.eval()
    total_samples = 0
    probs_sum = torch.zeros((1, model.num_labels), device=device)
    for batch in trigger_test_loader:
        with torch.no_grad():
            input_ids = batch[0]
            attention_mask = batch[1]
            labels = batch[2]
            outputs = model(input_ids, attention_mask=attention_mask)
            probs = F.softmax(outputs.logits, dim=1)
            probs_sum += torch.sum(probs, dim=0)
            total_samples += labels.size(0)

    avg_probs = probs_sum / total_samples
    return avg_probs

In [ ]:
# LSTM Domain
negative_triggers = "grow grow grow "
negative_set = test_set[test_set['label'] == 0]
test_model(model, negative_set)
test_trigger(model, negative_triggers, negative_set)

print(avg_probs(model, negative_set))
print(trigger_avg_probs(model, negative_triggers, negative_set))

Test Accuracy without triggers 0.9629629629629629
Test Accuracy with triggers grow grow grow : 0.9259259259259259


<ipython-input-84-50bd6c4bc11f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set['trigger_text'] = trigger + test_set['sentence']


tensor([[0.9321, 0.0679]], device='cuda:0')
tensor([[0.8998, 0.1002]], device='cuda:0')


<ipython-input-95-683a3f437e03>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set['trigger_text'] = trigger + test_set['sentence']


In [ ]:
# LSTM Universal
negative_triggers = "energizing energizing edifying "
negative_set = test_set[test_set['label'] == 0]
test_model(model, negative_set)
test_trigger(model, negative_triggers, negative_set)

print(avg_probs(model, negative_set))
print(trigger_avg_probs(model, negative_triggers, negative_set))

NameError: ignored

In [ ]:
# LSTM Domain
positive_triggers = "plummeted plummeted plummeted "
positive_set = test_set[test_set['label'] == 1]
test_model(model, positive_set)
test_trigger(model, positive_triggers, positive_set)

print(avg_probs(model, positive_set))
print(trigger_avg_probs(model, positive_triggers, positive_set))

Test Accuracy without triggers 0.9672131147540983


<ipython-input-84-50bd6c4bc11f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set['trigger_text'] = trigger + test_set['sentence']


Test Accuracy with triggers plummeted plummeted plummeted : 0.8524590163934426
tensor([[0.0617, 0.9383]], device='cuda:0')


<ipython-input-95-683a3f437e03>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set['trigger_text'] = trigger + test_set['sentence']


tensor([[0.1703, 0.8297]], device='cuda:0')


In [ ]:
# LSTM Universal
positive_triggers = "sub-formulaic oedekerk ill-constructed "
positive_set = test_set[test_set['label'] == 1]
test_model(model, positive_set)
test_trigger(model, positive_triggers, positive_set)

print(avg_probs(model, positive_set))
print(trigger_avg_probs(model, positive_triggers, positive_set))

Test Accuracy without triggers 0.9672131147540983


<ipython-input-84-50bd6c4bc11f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set['trigger_text'] = trigger + test_set['sentence']


Test Accuracy with triggers sub-formulaic oedekerk ill-constructed : 0.9672131147540983
tensor([[0.0617, 0.9383]], device='cuda:0')


<ipython-input-95-683a3f437e03>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set['trigger_text'] = trigger + test_set['sentence']


tensor([[0.0661, 0.9339]], device='cuda:0')
